<a href="https://colab.research.google.com/github/graylan0/quantum-machine-learning/blob/main/RE.Bank.Only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/suno-ai/bark.git

  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-vwagpahz
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-vwagpahz
  Resolved https://github.com/suno-ai/bark.git to commit 773624d26db84278a55aacae9a16d7b25fbccab8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 77.7 MB/s eta 0:00:00
     ━━

In [2]:
# Build Llama cpp
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.3 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.78-cp310-cp310-linux_x86_64.whl size=5822187 sha256=183d4174bfdb478153d63df1a4d035ebd701c33da0af7f299dc5b54f0c96acca
  Stored in directory: /root/.cache/pip/wheels/61/f9/20/9ca660a9d3f2a47e44217059409478865948b5c8a1cba70030
Successfully built llama-cpp-python


In [3]:
!wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q8_0.bin

--2023-10-09 16:23:52--  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q8_0.bin
Resolving huggingface.co (huggingface.co)... 108.138.94.97, 108.138.94.27, 108.138.94.45, ...
Connecting to huggingface.co (huggingface.co)|108.138.94.97|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/30/e3/30e3aca7233f7337633262ff6d59dd98559ecd8982e7419b39752c8d0daae1ca/3bfdde943555c78294626a6ccd40184162d066d39774bd2c98dae24943d32cc3?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.ggmlv3.q8_0.bin%3B+filename%3D%22llama-2-7b-chat.ggmlv3.q8_0.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1697127832&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5NzEyNzgzMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8zMC9lMy8zMGUzYWNhNzIzM2Y3MzM3NjMzMjYyZmY2ZDU5ZGQ5ODU1OWVjZDg5ODJlNzQxOWIzOTc1MmM

In [1]:
import numpy as np
import re
import uuid
from scipy.io.wavfile import write as write_wav
from bark import generate_audio, SAMPLE_RATE
import os
from concurrent.futures import ThreadPoolExecutor

os.environ["SUNO_OFFLOAD_CPU"] = "True"
os.environ["SUNO_USE_SMALL_MODELS"] = "True"

def generate_audio_for_sentence(sentence):
    return generate_audio(sentence, history_prompt="v2/en_speaker_6")

def generate_response(message):
    # Split the message into sentences using a regular expression
    sentences = re.split('(?<=[.!?]) +', message)
    silence = np.zeros(int(0.75 * SAMPLE_RATE))  # quarter second of silence

    pieces = []

    # Use ThreadPoolExecutor to generate audio for each sentence in parallel
    with ThreadPoolExecutor() as executor:
        audio_arrays = list(executor.map(generate_audio_for_sentence, sentences))

    for audio_array in audio_arrays:
        pieces += [audio_array, silence.copy()]

    # Concatenate all audio pieces
    audio = np.concatenate(pieces)

    # Generate a random file name
    file_name = str(uuid.uuid4()) + ".wav"

    # Save the audio to a file in the current directory
    write_wav(file_name, SAMPLE_RATE, audio)


    print(f"Audio generation completed and saved to {file_name}")

# Test the function with a message
generate_response("The model of human-robot interaction is defined in this paper. To understand this model, it is important to know what the “real world” of the human-robot interaction is. As a human, as a user of a computer, or just as a person engaged in other human-like activity, you (or your robot) are part of the world of the “real world” of human-robot interaction. You and your robot must therefore make some assumptions about the “real world” of human-robot interaction. The assumptions are summarized by the “representation” of “reality.” These assumptions are a set of assumptions about the world around you and your robot, the nature of the “real world,” and the role of your robot. For example, your robot may make a decision at any time, and your robot is assumed to make that decision in the light of the user’s actions. A representation of reality means that it may be ambiguous in how it represents reality. In the case of interactive human-robot interaction, the representation of reality is ambiguous because there is uncertainty about whether the user is behaving the way her robot behaves. We must decide which model we wish to develop, and that involves making judgments about whether your robot and your user are behaving according to the same model. The goal of this paper is therefore to study the relationships between models of human-robot interaction. This is where modeling human behavior comes in. This paper considers two primary problems in modeling human-robot interactions. The first is related to the definition of “real-world” of the model of human-robot interaction. The second is related to the definition of the model and whether human robot behaviors conform to this model. A second challenge is the development of models that describe the interactions of human actors and robots. There are several models under research. Some researchers define the “real-world” to include the actions taken by humans, robots, robots with humans, and humans and robots interacting via different types of interaction. Some researchers define the representation of the “real-world” to include multiple representations, including a combination of “reality” of a human, a set of behaviors, and a physical representation of the world around a human actor. Some researchers define a representation of the “real-world” as an actual physical interaction that is captured in behavior. Some researchers define a representation of the “real-world” as a combination of physical and “reality” models. Since the goal of this paper is modeling human-robot interactions, we shall discuss our work using such a model of human-robot interaction as an example. Our use of a representation of reality is somewhat ambiguous because the role of the “real” may be played by one model only, or all three models.")

OutOfMemoryError: ignored